In [1]:
from random import randint

In [2]:
def make_player_dict():
    to_return = {
        "money": 3,
        "wheat field": 1,
        "ranch": 0,
        "bakery": 1,
        "cafe": 0,
        "convenience store": 0,
        "forest": 0,
        "stadium": 0,
        "tv station": 0,
        "business center": 0,
        "cheese factory": 0,
        "furniture factory": 0,
        "mine": 0,
        "family restaurant": 0,
        "apple orchard": 0,
        "market": 0,
        "train station": 0,
        "shopping mall": 0,
        "amusement park": 0,
        "radio tower": 0
    }
    return to_return
def make_game_dict():
    to_return = {
        "wheat field": 6,
        "ranch": 6,
        "bakery": 6,
        "cafe": 6,
        "convenience store": 6,
        "forest": 6,
        "stadium": 6,
        "tv station": 0,
        "business center": 0,
        "cheese factory": 0,
        "furniture factory": 0,
        "mine": 0,
        "family restaurant": 0,
        "apple orchard": 0,
        "market": 0,
        "train station": 0,
        "shopping mall": 0,
        "amusement park": 0,
        "radio tower": 0
    }
    return to_return

In [3]:
def resolve_roll(roll,player_dicts,player_rolling):
    current_player_dict = player_dicts[player_rolling] # for convenience, we name the current player
    num_players = len(player_dicts) # how many players there are
    mall = current_player_dict['shopping mall'] # 0 or 1 if player has shopping mall
    if roll == 1: # wheat field case
        for player in player_dicts:
            player['money'] += player['wheat field'] # every player gets one money per owned wheat field
            
    elif roll == 2: # ranch and bakery
        for player in player_dicts:
            player['money'] += player['ranch'] # every player gets one money per owned ranch
        current_player_dict['money'] += current_player_dict['bakery'] * (1+mall) # rolling player gets money per bakery
        
    elif roll == 3: # bakery and cafe, cafe first
        for ii in range(1,num_players):
            player_looking_at = (player_rolling-ii)%num_players # payment is dolled out in reverse player order
            mall_for_red = player_dicts[player_looking_at]['shopping mall']
            num_cafes = player_dicts[player_looking_at]['cafe'] * (1+mall_for_red) # how many cafes does this player have
            money_got = min(num_cafes,current_player_dict['money']) # how much money does this player get
            player_dicts[player_looking_at]['money'] += money_got # add this money to the player
            current_player_dict['money'] -= money_got # take the money from the roller
        
        current_player_dict['money'] += current_player_dict['bakery'] * (1+mall) # rolling player gets bakery money
        
    elif roll == 4: # convenience store
        current_player_dict['money'] += current_player_dict['convenience store'] * (3+mall)
        
    elif roll == 5: # forest
        for player in player_dicts:
            player['money'] += player['forest']
            
    elif roll == 6: # stadium, tv station, business center
        pass # I'm leaving this for you
    
    elif roll == 7: # cheese factory
        pass # I'm leaving this for you
    
    elif roll == 8: # furniture factory
        num_gears = current_player_dict['forest'] + current_player_dict['mine']
        current_player_dict['money'] += 3*num_gears
    
    elif roll == 9: # mine and family restaurant
        pass # I'm leaving this for you
    
    elif roll == 10: # apple orchard
        pass # I'm leaving this for you
    
    elif roll >= 11: # fruit and veggie market
        pass # I'm leaving this for you
    
    return player_dicts

In [4]:
fake_players = [make_player_dict(),make_player_dict()]
for key in fake_players[0]:
    # give the players one of everything
    fake_players[0][key] = 1
    fake_players[1][key] = 1
for roll in range(1,13): #test every roll does not break
    fake_players = resolve_roll(roll,fake_players,0)
    print(roll,":",fake_players[0]['money'],",",fake_players[1]['money'])

1 : 2 , 2
2 : 5 , 3
3 : 5 , 5
4 : 9 , 5
5 : 10 , 6
6 : 10 , 6
7 : 10 , 6
8 : 16 , 6
9 : 16 , 6
10 : 16 , 6
11 : 16 , 6
12 : 16 , 6


In [5]:
def simulate_game(strategy_list):
    num_players = len(strategy_list) # determine how many players there are
    player_dicts = [make_player_dict() for player in num_players] # generate a player dictionary for each player
    player_rolling = randint(0,num_players-1) # determine who is up first by choosing a random number between 0 and num_players-1
    while True: # continue until someone wins
        player_advance = 1
        roll = randint(1,6) # roll the first die
        if player_dicts[player_rolling]["train station"] > 0:
            roll2 = randint(1,6)
            if player_dicts[player_rolling]['amusement park'] > 0 and roll == roll2:
                player_advance = 0
            roll += roll2 # roll the second die if the player has the train station
            
        player_dicts = resolve_roll(roll,player_dicts,player_rolling) # apply the roll
        
        for ii in range(num_players): # seeing if anyone has won
            player_looking_at = (player_rolling+ii)%num_players # we start with the player who rolled and work clockwise around
            if player_dicts[player_looking_at]['money'] > 40: # test if this player has won by these not quite right standards
                return player_looking_at
            
        player_dicts[player_rolling] = strategy(player_dicts[player_rolling]) # apply the strategy for card purchase
            
        player_rolling = (player_rolling+player_advance) % num_players # advance to the next player if no one has won yet